# Canonical task form

Under the hood, all Python, shell and workflow task definitions generated by the
`pydra.design.*.define` decorators/functions are translated to
[dataclasses](https://docs.python.org/3/library/dataclasses.html) by the
[Attrs](https://www.attrs.org/en/stable/). While the more compact syntax described
in the [Python-tasks](./4-python.html), [Shell-tasks](./5-shell.html) and [Workflow](./6-workflow.html)
tutorials is convenient when designing tasks for specific use cases, it is too magical
for linters follow. Therefore, when designing task definitions to be used by third
parties (e.g. `pydra-fsl`, `pydra-ants`) it is recommended to favour the, more
explicit, "canonical" dataclass form.

The syntax of the canonical form is close to that used by the
[Attrs](https://www.attrs.org/en/stable/) package itself, with class type annotations
used to define the fields of the inputs and outputs of the task. Tasks defined in canonical
form will be able to be statically type-checked by [MyPy](https://mypy-lang.org/).

## Python-task definitions

Python tasks in dataclass form are decorated by `pydra.design.python.define`
with inputs listed as type annotations. Outputs are similarly defined in a nested class
called `Outputs`. The function to be executed should be a staticmethod called `function`.
Default values can also be set directly, as with Attrs classes.

In order to allow static type-checkers to check the type of outputs of tasks added
to workflows, it is also necessary to explicitly extend from the `pydra.engine.specs.PythonDef`
and `pydra.engine.specs.PythonOutputs` classes (they are otherwise set as bases by the
`define` method implicitly). Thus the "canonical form" of Python task definition is as
follows

In [1]:
from pprint import pprint
from pydra.engine.helpers import fields_dict
from pydra.engine.specs import PythonDef, PythonOutputs
from pydra.design import python


@python.define
class CanonicalPythonDef(PythonDef["CanonicalPythonDef.Outputs"]):
    """Canonical Python task definition class for testing

    Args:
        a: First input
            to be inputted
        b: Second input
    """

    a: int
    b: float = 2.0  # set default value

    class Outputs(PythonOutputs):
        """
        Args:
            c: Sum of a and b
            d: Product of a and b
        """

        c: float
        d: float

    @staticmethod
    def function(a, b):
        return a + b, a / b


pprint(fields_dict(CanonicalPythonDef))
pprint(fields_dict(CanonicalPythonDef.Outputs))

{'a': arg(name='a', type=<class 'int'>, default=NO_DEFAULT, help='First input to be inputted', requires=[], converter=None, validator=None, hash_eq=False, allowed_values=frozenset(), copy_mode=<CopyMode.any: 15>, copy_collation=<CopyCollation.any: 0>, copy_ext_decomp=<ExtensionDecomposition.single: 1>, readonly=False),
 'b': arg(name='b', type=<class 'float'>, default=2.0, help='Second input', requires=[], converter=None, validator=None, hash_eq=False, allowed_values=frozenset(), copy_mode=<CopyMode.any: 15>, copy_collation=<CopyCollation.any: 0>, copy_ext_decomp=<ExtensionDecomposition.single: 1>, readonly=False),
 'function': arg(name='function', type=typing.Callable, default=<function CanonicalPythonDef.function at 0x7f5b9086f420>, help='', requires=[], converter=None, validator=None, hash_eq=True, allowed_values=frozenset(), copy_mode=<CopyMode.any: 15>, copy_collation=<CopyCollation.any: 0>, copy_ext_decomp=<ExtensionDecomposition.single: 1>, readonly=False)}
{'c': out(name='c', t

To set additional attributes other than the type and default, such as `allowed_values`
and `validators`, `python.arg` and `python.out` can be used instead.

In [2]:
import attrs.validators


@python.define
class CanonicalPythonDef(PythonDef["CanonicalPythonDef.Outputs"]):
    """Canonical Python task definition class for testing

    Args:
        a: First input
            to be inputted
        b: Second input
    """

    a: int = python.arg(allowed_values=[1, 2, 3, 4, 5])
    b: float = python.arg(default=2.0, validator=attrs.validators.not_(0))

    class Outputs(PythonOutputs):
        """
        Args:
            c: Sum of a and b
            d: Product of a and b
        """

        c: float
        d: float

    @staticmethod
    def function(a, b):
        return a + b, a / b


pprint(fields_dict(CanonicalPythonDef))
pprint(fields_dict(CanonicalPythonDef.Outputs))

{'a': arg(name='a', type=<class 'int'>, default=NO_DEFAULT, help='First input to be inputted', requires=[], converter=None, validator=None, hash_eq=False, allowed_values=frozenset({1, 2, 3, 4, 5}), copy_mode=<CopyMode.any: 15>, copy_collation=<CopyCollation.any: 0>, copy_ext_decomp=<ExtensionDecomposition.single: 1>, readonly=False),
 'b': arg(name='b', type=<class 'float'>, default=2.0, help='Second input', requires=[], converter=None, validator=<not_ validator wrapping 0, capturing (<class 'ValueError'>, <class 'TypeError'>)>, hash_eq=False, allowed_values=frozenset(), copy_mode=<CopyMode.any: 15>, copy_collation=<CopyCollation.any: 0>, copy_ext_decomp=<ExtensionDecomposition.single: 1>, readonly=False),
 'function': arg(name='function', type=typing.Callable, default=<function CanonicalPythonDef.function at 0x7f5b8ad7ad40>, help='', requires=[], converter=None, validator=None, hash_eq=True, allowed_values=frozenset(), copy_mode=<CopyMode.any: 15>, copy_collation=<CopyCollation.any: 0

## Shell-task definitions

The canonical form of shell tasks is the same as for Python tasks, except a string `executable`
attribute replaces the `function` staticmethod.

In [3]:
import os
from pathlib import Path
from fileformats import generic
from pydra.design import shell
from pydra.engine.specs import ShellDef, ShellOutputs
from pydra.utils.typing import MultiInputObj


@shell.define
class CpWithSize(ShellDef["CpWithSize.Outputs"]):

    executable = "cp"

    in_fs_objects: MultiInputObj[generic.FsObject]
    recursive: bool = shell.arg(argstr="-R")
    text_arg: str = shell.arg(argstr="--text-arg")
    int_arg: int | None = shell.arg(argstr="--int-arg")
    tuple_arg: tuple[int, str] | None = shell.arg(argstr="--tuple-arg")

    class Outputs(ShellOutputs):

        @staticmethod
        def get_file_size(out_file: Path) -> int:
            """Calculate the file size"""
            result = os.stat(out_file)
            return result.st_size

        copied: generic.FsObject = shell.outarg(path_template="copied")
        out_file_size: int = shell.out(callable=get_file_size)


pprint(fields_dict(CpWithSize))
pprint(fields_dict(CpWithSize.Outputs))

{'additional_args': arg(name='additional_args', type=list[str | fileformats.generic.file.File], default=Factory(factory=<class 'list'>, takes_self=False), help='Additional free-form arguments to append to the end of the command.', requires=[], converter=<function state_array_support.<locals>.state_array_wrapper at 0x7f5b8ad6b9c0>, validator=None, hash_eq=False, copy_mode=<CopyMode.any: 15>, copy_collation=<CopyCollation.any: 0>, copy_ext_decomp=<ExtensionDecomposition.single: 1>, readonly=False, argstr='', position=None, sep=' ', allowed_values=None, container_path=False, formatter=None),
 'copied': outarg(name='copied', type=<class 'fileformats.generic.fsobject.FsObject'>, default=NO_DEFAULT, help='', requires=[], converter=None, validator=None, hash_eq=False, copy_mode=<CopyMode.any: 15>, copy_collation=<CopyCollation.any: 0>, copy_ext_decomp=<ExtensionDecomposition.single: 1>, readonly=False, argstr='', position=6, sep=' ', allowed_values=None, container_path=False, formatter=None, 

## Workflow definitions

Workflows can also be defined in canonical form, which is the same as for Python tasks
but with a staticmethod called `constructor` that constructs the workflow.

In [4]:
from pydra.design import python, workflow
from pydra.engine.specs import WorkflowDef, WorkflowOutputs


# Example python task definitions
@python.define
def Add(a, b):
    return a + b


@python.define
def Mul(a, b):
    return a * b


@workflow.define
class CanonicalWorkflowDef(WorkflowDef["CanonicalWorkflowDef.Outputs"]):

    @staticmethod
    def a_converter(value):
        if value is None:
            return value
        return float(value)

    a: int
    b: float = workflow.arg(
        help="A float input",
        converter=a_converter,
    )

    @staticmethod
    def constructor(a, b):
        add = workflow.add(Add(a=a, b=b))
        mul = workflow.add(Mul(a=add.out, b=b))
        return mul.out

    class Outputs(WorkflowOutputs):
        out: float